### 使用python进行文本分类

#### 1.准备数据：从文本中构建词向量

In [58]:
#词表到向量的转换函数
def loadDataSet():
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],                #切分的词条
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]           #1代表侮辱性文字，0代表正常言论
    return postingList,classVec

def createVocabList(dataSet):
    vocabSet = set([])   #创建一个空集
    for document in dataSet:
        vocabSet = vocabSet|set(document)  #创建两个集合的并集
    return list(vocabSet)

def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)    #创建一个其中所含元素都为0的向量
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else: pring("the word:%s is not in my Vocabulary!" % word)
    return returnVec
    

 #### 2.训练算法：从词向量计算概率

In [59]:
import numpy as np
"""
函数说明:朴素贝叶斯分类器训练函数

Parameters:
    trainMatrix - 文档矩阵
    trainCategory - 每篇文档类别标签所构成的向量
Returns:
    p0Vect - 非侮辱类的条件概率数组
    p1Vect - 侮辱类的条件概率数组
    pAbusive - 文档属于侮辱类的概率
"""
def trainNB0(trainMatrix, trainGategory):
    numTrainDocs = len(trainMatrix)   #返回训练的文档数目
    numWords = len(trainMatrix[0])   #返回每一篇文档的词条数
    pAbusive = sum(trainGategory)/float(numTrainDocs)  #文档属于侮辱类的概率
    p0Num = np.ones(numWords); p1Num = np.ones(numWords)  #创建numpy.zeros数组,词条出现数初始化为0
    p0Denom = 2.0; p1Denom = 2.0   #分母初始化为0
    for i in range(numTrainDocs):
        if trainGategory[i] == 1:  #统计属于侮辱类的条件概率所需的数据，即P(w0|1),P(w1|1),P(w2|1)···
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:  #统计属于非侮辱类的条件概率所需的数据，即P(w0|0),P(w1|0),P(w2|0)···
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num/p1Denom)
    p0Vect = np.log(p0Num/p0Denom)
    return p0Vect,p1Vect,pAbusive  #返回属于非侮辱类的条件概率数组，属于侮辱类的条件概率数组，文档属于侮辱类的概率


#### 3.测试算法：根据现实情况修改分类器

In [60]:
"""
函数说明:朴素贝叶斯分类函数

Parameters:
    vec2Classify - 待分类的词条数组
    p0Vec - 非侮辱类的条件概率数组
    p1Vec - 侮辱类的条件概率数组
    pClass1 - 文档属于侮辱类的概率
Returns:
    类别标签 - 1或0
"""
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify*p1Vec)+np.log(pClass1)
    p0 = sum(vec2Classify*p0Vec)+np.log(1.0-pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V,p1V,pAb = trainNB0(np.array(trainMat),np.array(listClasses))
    testEntry = ['love','my','dalmation']
    thisDoc = np.array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'classified as:',classifyNB(thisDoc,p0V,p1V,pAb))
    testEntry = ['stupid','garbage']
    thisDoc = np.array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'classified as:',classifyNB(thisDoc,p0V,p1V,pAb))

#### 4.准备数据：文档词袋模型

In [61]:
#词袋模型
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

### 示例：使用朴素贝叶斯过滤垃圾邮件

In [65]:
#文件解析及完整的垃圾邮件测试函数
def textParse(bigString):  #接收一个大字符串并将其解析为字符串列表
    import re
    listOfTokens = re.split(r'\W*',bigString)  #利用正则表达式来切分句子
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]  #将字符串全部转换为小写，并去除空格以及长度小于3的字符串

def spamTest():
    docList=[]; classList=[]; fullText=[]
    for i in range(1,26):   #遍历25个文件，range(1,26)代表从1到26，不包含26.
        wordList = textParse(open('data/email/spam/%d.txt'%i).read()) #读取每个垃圾邮件，并将字符串转换成字符串列表
        docList.append(wordList)
        #fullText.extend(wordList)  #不明白用来干嘛的？注释掉也没有影响
        classList.append(1)  #标记垃圾邮件，1表示垃圾文件
        wordList = textParse(open('data/email/ham/%d.txt' % i).read())  #读取每个非垃圾邮件，并将字符串转换成字符串列表
        docList.append(wordList)
        #fullText.extend(wordList)#不明白用来干嘛的？注释掉也没有影响
        classList.append(0)  #标记非垃圾邮件，0表示非垃圾文件
    vocabList = createVocabList(docList)  #创建词汇表，不重复
    trainingSet = list(range(50));  testSet=[]  #创建存储训练集的索引值的列表和测试集的索引值的列表
    for i in range(10):  #从50个邮件中，随机挑选出40个作为训练集,10个做测试集
        randIndex = int(np.random.uniform(0,len(trainingSet)))  #随机选取索索引值
        testSet.append(trainingSet[randIndex])  #添加测试集的索引值
        del(trainingSet[randIndex])  #在训练集列表中删除添加到测试集的索引值
    trainMat = []; trainClasses = []  #创建训练集矩阵和训练集类别标签系向量
    for docIndex in trainingSet:  #遍历训练集
        trainMat.append(setOfWords2Vec(vocabList,docList[docIndex]))  #将生成的词集模型添加到训练矩阵中
        trainClasses.append(classList[docIndex])  #将类别添加到训练集类别标签系向量中
    p0V,p1V,pSpam = trainNB0(np.array(trainMat),np.array(trainClasses))  #训练朴素贝叶斯模型
    errorCount = 0  #错误分类计数
    for docIndex in testSet:  #遍历测试集
        wordVector = setOfWords2Vec(vocabList,docList[docIndex])  #测试集的词集模型
        if classifyNB(np.array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:  #如果分类错误
            errorCount +=1  #错误计数加1
            print(' classification error:',docList[docIndex])
    print(' the error rate is:', float(errorCount)/len(testSet))

In [63]:
spamTest()

 classification error: ['home', 'based', 'business', 'opportunity', 'knocking', 'your', 'door', 'don抰', 'rude', 'and', 'let', 'this', 'chance', 'you', 'can', 'earn', 'great', 'income', 'and', 'find', 'your', 'financial', 'life', 'transformed', 'learn', 'more', 'here', 'your', 'success', 'work', 'from', 'home', 'finder', 'experts']
 the error rate is: 0.1


C:\ProgramData\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [64]:
spamTest()

 classification error: ['scifinance', 'now', 'automatically', 'generates', 'gpu', 'enabled', 'pricing', 'risk', 'model', 'source', 'code', 'that', 'runs', '300x', 'faster', 'than', 'serial', 'code', 'using', 'new', 'nvidia', 'fermi', 'class', 'tesla', 'series', 'gpu', 'scifinance', 'derivatives', 'pricing', 'and', 'risk', 'model', 'development', 'tool', 'that', 'automatically', 'generates', 'and', 'gpu', 'enabled', 'source', 'code', 'from', 'concise', 'high', 'level', 'model', 'specifications', 'parallel', 'computing', 'cuda', 'programming', 'expertise', 'required', 'scifinance', 'automatic', 'gpu', 'enabled', 'monte', 'carlo', 'pricing', 'model', 'source', 'code', 'generation', 'capabilities', 'have', 'been', 'significantly', 'extended', 'the', 'latest', 'release', 'this', 'includes']
 the error rate is: 0.1


C:\ProgramData\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
